# 异步IO-Coroutine和Task

In [1]:
import asyncio
import time
import datetime

## Coroutine

通过`async`和`await`实现asyncio应用。

In [2]:
async def hello():
    print('Hello')
    await asyncio.sleep(1)
    print('World!')

`Croutine`对象

In [3]:
hello()

<coroutine object hello at 0x7ffd403fd830>

In [4]:
await hello()

Hello
World!


In [5]:
async def say_after(delay: int, message: str):
    await asyncio.sleep(delay)
    print(message)

In [6]:
async def main():
    print(f'Start at {time.strftime("%X")}')
    await say_after(1, 'Hello')
    await say_after(2, 'World!')
    
    # 运行时间3秒
    print(f'Finished at {time.strftime("%X")}')

In [7]:
await main()

Start at 14:21:20
Hello
World!
Finished at 14:21:23


`asyncio.create_task`将`coroutine`作为`Task`并行运行。

In [8]:
async def main():
    task1 = asyncio.create_task(say_after(1, 'Hello'))
    task2= asyncio.create_task(say_after(2, 'World!'))
    
    print(f'Start at {time.strftime("%X")}')
    await task1
    await task2
    
    # 运行时间2秒
    print(f'Finished at {time.strftime("%X")}')

In [9]:
await main()

Start at 14:21:23
Hello
World!
Finished at 14:21:25


## Awaitables

当对象是`Awaitable`对象时，可以通过`await`表达式。

三种主要Awaitablel类型：`Coroutine`、`Task`和`Future`。

### Coroutine

In [10]:
async def nested():
    return 42

In [11]:
nested()

<coroutine object nested at 0x7ffd28a549e0>

* `async def` 定义`coroutine`函数
* 调用`couroutine`函数，返回`coroutine`对象

### Task

Task用来并发的调度`Coroutine`.

通过`asyncio.create_taks()`可以将`Coroutine`封装成`Task`，`coroutine`调度后自动执行。

In [12]:
task = asyncio.create_task(nested())
await task

42

### Future

Future是底层`awaitable`对象，表示异步操作的结果。

In [13]:
async def delay(seconds: int, future: asyncio.Future):
    await asyncio.sleep(seconds)
    future.set_result('200')


async def main():
    loop = asyncio.get_event_loop()
    future = loop.create_future()
    asyncio.create_task(delay(1, future))

    # wait future done.
    await future
    print(f'future done, result {future.result()}')


In [14]:
await main()

future done, result 200
